In [129]:
#!pip install yargy

In [130]:
import pandas as pd
import numpy as np

from yargy.tokenizer import MorphTokenizer
from yargy import (
    Parser,
    or_, rule
)
from yargy.pipelines import morph_pipeline
from yargy.predicates import (
    eq, gram
)

In [131]:
data = pd.read_csv('test_data.csv') #/content/drive/MyDrive/Bewise.ai/

In [132]:
data.head()

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага


In [133]:
print('Count speeching =', len(data['dlg_id'].unique()))

Count speeching = 6


В тестовом датасете 6 диалогов различной длительности

## Извлечение сущностей

### Yargy

In [134]:
# Yargy

In [135]:
# из дата фрейма возьмем только реплики менеджеров
# т.к реплики приветствия и прощания менеджеров стандартизированы правилами компании
# Приветствие и имя явно находятся в начале диалога, поэтому при работе функции
# можно парсить только первые 3 строчки (в коде прорабатывается весь диалог на случай "поводных камней").
# (Диалоги разной длительности)
# Аналогично с окончанием разговора. 

manager_yargy = {}

for i, group in data.groupby(by=['dlg_id', 'role']):
  if i[1] == 'manager':
    manager_yargy[i[0]] = list(group['text'])
  print(f'В диалоге {i}, количество реплик = {len(group)}')  

В диалоге (0, 'client'), количество реплик = 93
В диалоге (0, 'manager'), количество реплик = 16
В диалоге (1, 'client'), количество реплик = 26
В диалоге (1, 'manager'), количество реплик = 29
В диалоге (2, 'client'), количество реплик = 45
В диалоге (2, 'manager'), количество реплик = 40
В диалоге (3, 'client'), количество реплик = 31
В диалоге (3, 'manager'), количество реплик = 22
В диалоге (4, 'client'), количество реплик = 21
В диалоге (4, 'manager'), количество реплик = 14
В диалоге (5, 'client'), количество реплик = 63
В диалоге (5, 'manager'), количество реплик = 80


### RULES

In [136]:
# Name

NAME = or_(
    rule(
        morph_pipeline(['звать']),
        gram('Name')
        ),

    rule(
        gram('Name'),
        morph_pipeline(['звать'])
        )
      )

# Company

COMPANY = or_(
    rule(
        morph_pipeline(['компания']),
        gram('NOUN'),
        gram('NOUN').optional()
    ),
    rule(
        morph_pipeline(['компания']),
        gram('NOUN'),
        eq('бизнес').optional()
    ),

    rule(
        morph_pipeline(['компания']),
        eq('диджитал').optional(),
        gram('NOUN')
        )
    )

# Приветствие - прощание

HELLO = morph_pipeline([
    'Здравствуй',
    'Здравствуйте',
    'Добрый день',
    'Доброе утро',
    'Привет',
    'Добрый вечер',
    'Добрый'
])

GOOD_BYE = morph_pipeline([
    'До свидания',
    'Приятного вечера',
    'Всего доброго',
    'Хорошего дня',
    'Хорошего вечера'
])

### Extractor

In [137]:
def extractor(dialog, rules):
  
  parser = Parser(rules)

  for replica in dialog:
      matches = list(parser.findall(replica))
      if matches and rules not in (NAME, COMPANY):
        return replica
      elif matches:
        for match in matches:
          toks = ' '.join([_.value for _ in match.tokens])
          return [toks, replica]

## Итоговый датафрейм

In [138]:
# Таблица id, приветствие, представление, Имя, Компания, Прощание

cosolidation_table = pd.DataFrame(index=manager_yargy)

In [139]:
for i in range(len(manager_yargy)):

  # hello
  if extractor(manager_yargy[i], HELLO):
    cosolidation_table.loc[i, 'hello'] = extractor(manager_yargy[i], HELLO)

  # extract name + replica
  if extractor(manager_yargy[i], NAME):
    cosolidation_table.loc[i, 'replica_with_name'] = extractor(manager_yargy[i], NAME)[1]
    cosolidation_table.loc[i, 'name'] = extractor(manager_yargy[i], NAME)[0]

  # extract name_company (есть возможность вывести так же реплику)
  if extractor(manager_yargy[i], COMPANY):
    cosolidation_table.loc[i, 'name_company'] = extractor(manager_yargy[i], COMPANY)[0]

  # bye
  if extractor(manager_yargy[i], GOOD_BYE):
    cosolidation_table.loc[i, 'bye'] = extractor(manager_yargy[i], GOOD_BYE)

  # greeting
  if extractor(manager_yargy[i], HELLO) and extractor(manager_yargy[i], GOOD_BYE):
    cosolidation_table.loc[i, 'greeting'] = True
  else: cosolidation_table.loc[i, 'greeting'] = False

In [140]:
cosolidation_table

,hello,replica_with_name,name,name_company,bye,greeting
0,Алло здравствуйте,Меня зовут ангелина компания диджитал бизнес з...,зовут ангелина,компания диджитал бизнес,Всего хорошего до свидания,True
1,Алло здравствуйте,Меня зовут ангелина компания диджитал бизнес з...,зовут ангелина,компания диджитал бизнес,До свидания,True
2,Алло здравствуйте,Меня зовут ангелина компания диджитал бизнес з...,зовут ангелина,компания диджитал бизнес,NaN,False
3,Алло дмитрий добрый день,Добрый меня максим зовут компания китобизнес у...,максим зовут,компания китобизнес,Угу все хорошо да понедельника тогда всего доб...,True
4,NaN,NaN,NaN,NaN,Во вторник все ну с вами да тогда до вторника ...,False
5,NaN,NaN,NaN,NaN,Ну до свидания хорошего вечера,False


In [141]:
cosolidation_table.to_csv('cosolidation_table.csv') #/content/drive/MyDrive/Bewise.ai/